In [1]:
import time
import pandas as pd
from kafka import KafkaConsumer

# Create a KafkaConsumer instance
consumer = KafkaConsumer(
    bootstrap_servers=['localhost:9091'], 
    auto_offset_reset='latest', 
    enable_auto_commit=False
)

# Subscribe to a specific topic
consumer.subscribe(topics=['foobar'])
N_sample = 30000 - 100
lst_delay = []
while True:
    msg_poll = consumer.poll(timeout_ms=0)
    for k, v in msg_poll.items():
        for msg in v: 
            topic = msg.topic
            partition = msg.partition
            offset = msg.offset
            timestamp = msg.timestamp
            value_msg = msg.value
            current_time = time.time()
            try:
                msg_time = float(value_msg.decode('utf-8').split(' ')[0])
                delay_ms = (current_time - msg_time) * 1000
            # print(f"Topic: {topic} | Partition: {partition} | Offset: {offset}  | Value: {value_msg.decode('utf-8')} | Time: {msg_time} | Delay: {delay}")
                lst_delay.append(delay_ms)
                # print(f"Delay: {delay_ms} ms")
            except Exception as e:
                # print(f"Error: {e}")
                continue
    if len(lst_delay) > N_sample:
        break

df_delay = pd.DataFrame(lst_delay, columns=['delay'])
df_delay.describe()

,delay
count,29920.000000
mean,5.535370
std,3.705196
min,1.297951
25%,3.607035
50%,4.721880
75%,6.191254
max,29.899836


In [4]:
df_delay[df_delay['delay'] > 10]

,delay
0,26.403904
1,26.372194
2,26.335955
3,26.263714
4,26.216984
...,...
27053,28.698206
27054,28.592110
27055,28.475046
27056,28.432131


In [5]:
msg_time, x, y = value_msg.decode('utf-8').split(' ')[:3]
msg_time = float(msg_time)
x = int(x)
y = int(y)
print(msg_time, x, y)

1730432977.004143 4 4
